* データマイニング特別演習
* 気象庁からダウンロードしたさいたま市の気温データを分析

In [ ]:
# google colab の場合は以下のコメントアウトを外す
#!wget https://github.com/KHiraGit/sudspg_ds/raw/main/dm02_data.zip
#!unzip -d data dm02_data.zip

In [ ]:
# 利用するライブラリをインポート
import os
import pandas as pd

In [ ]:
csv_file = os.path.join('data', 'data-2022.csv') # 読み込むファイルを設定
df = pd.read_csv(csv_file, encoding='shift-jis') # CSVファイルの文字コードを指定して読み込む
df.head()

In [ ]:
# 不要な行を省略したデータフレームを作成
csv_file = os.path.join('data', 'data-2022.csv') # 読み込むファイルを設定
df = pd.read_csv(csv_file, 
                 header=3, # 3行目までを無視、4行目を列名として設定
                 names=['年月日時', '気温', '品質情報', '均質番号'], # 列名を設定
                 encoding='shift-jis')
df['年月日時'] = pd.to_datetime(df['年月日時']) # '年月日時' を datetime型に変換
df.head()

In [ ]:
# 気温の折れ線グラフを作成
import matplotlib.pyplot as plt

df['気温'].plot(figsize=(15,5))

In [ ]:
# 気温のヒストグラムを作成
df['気温'].hist(figsize=(15,5))

In [ ]:
# 35度以上のデータを抽出
df_35 = df[df['気温'] >= 35] # 条件指定で抽出
print(len(df_35)) # df_35 の行数を表示
print(df_35.head(5)) # df_35 の先頭5行を表示

In [ ]:
import glob
import re

# glob 関数を使って dataフォルダの csvファイルのリストを作成 (年でソート)
files = sorted(glob.glob(os.path.join('data', '*.csv')))
for file in files: # リストのファイルを１つずつ順に処理
    print(file) # ファイル名を表示

In [ ]:
# 棒グラフ作成用のリストを準備 - 年ごとの状況
years = []
hours_35 = []
for file in files: # リストのファイルを１つずつ順に処理
    # ファイル名から 年 を抽出
    m = re.search(r'(\d\d\d\d)', file) # 数字が4つ連続する部分を抽出
    year = m.group(1)
    years.append(year)
    # 不要な行を省略したデータフレームを作成
    df = pd.read_csv(file, 
                    header=3, # 3行目までを無視、4行目を列名として設定
                    names=['年月日時', '気温', '品質情報', '均質番号'], # 列名を設定
                    encoding='shift-jis')
    # 35度以上のデータ数をリストに追加
    hour_35 = len(df[df['気温'] >= 35]) # 条件指定で抽出した日数
    hours_35.append(hour_35) # リストに追加
    # ファイル名、年、35度以上の日数を表示
    print(file, year, hour_35)
    
# 35度以上の日数の棒グラフを表示
plt.figure(figsize=(6,4)) # サイズを指定
plt.grid(axis='y') # 水平グリッド線を表示
plt.bar(years, hours_35) # 棒グラフを作成
plt.show() # グラフを表示

In [ ]:
# 棒グラフ作成用のリストを準備 - 月日ごとの状況
df_hour = pd.DataFrame() # 空のデータフレームを準備
for file in files: # リストのファイルを１つずつ順に処理
    # 不要な行を省略したデータフレームを作成
    df = pd.read_csv(file, 
                    header=3, # 3行目までを無視、4行目を列名として設定
                    names=['年月日時', '気温', '品質情報', '均質番号'],
                    encoding='shift-jis')
    df['年月日時'] = pd.to_datetime(df['年月日時']) # '年月日時' を datetime型に変換
    df_hour = df_hour.append(df) # データフレームを追加

dates = [] # 7月1日から9月30日までの日付 (x軸用)
dates_35 = [] # 7月1日から9月30日までの各日で35度以上のデータ数をカウント
for dd in range(1,32): # 7月
    dates.append(f'7{dd:02d}') # 指定された時刻で、35度以上のデータ数をカウント
    dates_35.append(len(df_hour[(df_hour['年月日時'].dt.month==7) & (df_hour['年月日時'].dt.day==dd) & (df_hour['気温'] >= 35)]))
for dd in range(1,32): # 8月
    dates.append(f'8{dd:02d}') # 指定された時刻で、35度以上のデータ数をカウント
    dates_35.append(len(df_hour[(df_hour['年月日時'].dt.month==8) & (df_hour['年月日時'].dt.day==dd) & (df_hour['気温'] >= 35)]))
for dd in range(1,31): # 9月
    dates.append(f'9{dd:02d}') # 指定された時刻で、35度以上のデータ数をカウント
    dates_35.append(len(df_hour[(df_hour['年月日時'].dt.month==9) & (df_hour['年月日時'].dt.day==dd) & (df_hour['気温'] >= 35)]))

# 各時間帯の35度以上のデータ数を棒グラフを表示
plt.figure(figsize=(18,4)) # サイズを指定
plt.grid(axis='y') # 水平グリッド線を表示
plt.bar(dates, dates_35) # 棒グラフを作成
plt.xticks(rotation=90) # x軸のラベルを縦書きに
plt.show() # グラフを表示

In [ ]:
# 棒グラフ作成用のリストを準備 - 時間帯ごとの状況
df_hour = pd.DataFrame() # 空のデータフレームを準備
for file in files: # リストのファイルを１つずつ順に処理
    # 不要な行を省略したデータフレームを作成
    df = pd.read_csv(file, 
                    header=3, # 3行目までを無視、4行目を列名として設定
                    names=['年月日時', '気温', '品質情報', '均質番号'],
                    encoding='shift-jis')
    df['年月日時'] = pd.to_datetime(df['年月日時']) # '年月日時' を datetime型に変換
    df_hour = df_hour.append(df) # データフレームを追加

hours = []
hours_35 = []
for hh in range(24):
    hours.append(hh) # 指定された時刻で、35度以上のデータ数をカウント
    hours_35.append(len(df_hour[(df_hour['年月日時'].dt.hour==hh) & (df_hour['気温'] >= 35)]))

# 各時間帯の35度以上のデータ数を棒グラフを表示
plt.figure(figsize=(6,4)) # サイズを指定
plt.grid(axis='y') # 水平グリッド線を表示
plt.bar(hours, hours_35) # 棒グラフを作成
plt.show() # グラフを表示

In [ ]:
# 35度以上の継続時間を確認
import csv

high_35_dict = {}
for file in files: # リストのファイルを１つずつ順に処理
    with open(file, encoding='shift-jis') as f:
        reader = csv.reader(f)
        high_35 = 0
        for row in reader:
            if len(row) == 0 or not re.search(r'^\d\d\d\d', row[0]) or not re.search(r'^\d\d\.\d', row[1]):
                continue # 空行、データ行以外、欠損値の行はスキップ
            if float(row[1]) >= 35.0:
                high_35 = high_35 + 1
            elif high_35 > 0 and float(row[1]) < 35.0:
                high_35_dict[row[0]] = high_35 # 35度未満になった時刻とそれ以前の継続時間を辞書形式で記録
                high_35 = 0

sorted_dict = dict(sorted(high_35_dict.items(), key=lambda t: t[1], reverse=True))
for key in sorted_dict.keys():
    if sorted_dict[key] < 5: # 継続時間が5時間以上のデータを表示
        break
    else:
        print(key, sorted_dict[key])
